<h1> Image detection comparasion

<h3> Importing required libraries

In [1]:
import cv2
import tensorflow as tf
import os
import numpy as np

/home/jupyterlab/conda/envs/python/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/jupyterlab/conda/envs/python/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/jupyterlab/conda/envs/python/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/jupyterlab/conda/envs/python/lib/python3

In [2]:
#!unzip archive.zip

In [3]:
#constants
IMG_WIDTH =50
IMG_HEIGHT =50
NUM_CATEGORIES = 5
EPOCHS = 10

<h3> Importing data

Data will be imported as first. It is divided directly in the training set and test set as it is already divided.

In [4]:
def get_data_np(path, size_width = IMG_WIDTH, size_height =  IMG_HEIGHT):
    images = list()
    labels = list()

    complete_path = os.getcwd() + path
    for folder in os.scandir(complete_path):
        #print(f"Scanninng folder {folder.name}")
        label = folder.name
        print(f"Reading folder: {folder.name}")
        for file in os.scandir(folder):
            if file.name == '.ipynb_checkpoints': continue
            # Reading each image
            image = cv2.imread(file.path)
            if image is None: print(f"Reading image {complete_path + file.name} was not possible")

            #Resizing the image to IMG_WIDTH, IMG_HEIGHT
            image = cv2.resize(image,(size_width,size_height))

            images.append(image)
            labels.append(label)
            print(f"Total images loaded: {len(labels)}", end='\r')
        print('\n')
    images = np.array(images)
    labels = np.array(labels)
    return images, labels        
    

In [5]:
images_train, labels_train_categories = get_data_np('/animals/train/')
images_test, labels_test_categories = get_data_np('/animals/val/')

Reading folder: cat
Total images loaded: 2737

Reading folder: dog
Total images loaded: 5364

Reading folder: elephant
Total images loaded: 8094

Reading folder: horse
Total images loaded: 10799

Reading folder: lion
Total images loaded: 13474

Reading folder: cat
Total images loaded: 300

Reading folder: dog
Total images loaded: 600

Reading folder: elephant
Total images loaded: 899

Reading folder: horse
Total images loaded: 1199

Reading folder: lion
Total images loaded: 1497



In [6]:
labels_train_categories

array(['cat', 'cat', 'cat', ..., 'lion', 'lion', 'lion'], dtype='<U8')

In [7]:
labels_test_categories

array(['cat', 'cat', 'cat', ..., 'lion', 'lion', 'lion'], dtype='<U8')

Let's now create a dictionary to map for each catergory

In [8]:
np.unique(labels_train_categories)

array(['cat', 'dog', 'elephant', 'horse', 'lion'], dtype='<U8')

In [9]:
mapping_categories = dict()
for i, label in enumerate(np.unique(labels_train_categories)):
    mapping_categories[label] = i

In [10]:
mapping_categories

{'cat': 0, 'dog': 1, 'elephant': 2, 'horse': 3, 'lion': 4}

In [11]:
def map_labels(labels):
    labels_categories = np.empty(labels.shape)
    
    for i, label in enumerate(labels):
        labels_categories[i] = mapping_categories[label]
    
    return labels_categories

In [12]:
labels_train = map_labels(labels_train_categories)
labels_test = map_labels(labels_test_categories)

In [13]:
labels_train

array([0., 0., 0., ..., 4., 4., 4.])

In [14]:
labels_test

array([0., 0., 0., ..., 4., 4., 4.])

<h3> Simple Model: Convolution with One layer

The first model to be built will be a simple convolutional netwcork with only a hidden layer.

In [15]:
model1 =  model = tf.keras.models.Sequential()

#Create a Conv2D layer
model1.add(tf.keras.layers.Conv2D(50, (3,3), activation='relu', input_shape=(IMG_WIDTH, IMG_HEIGHT, 3)))

#Create a Pooling layer
model1.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))

#Flatten layer
model1.add(tf.keras.layers.Flatten())

model1.add(tf.keras.layers.Dropout(0.05))

#Output layer
model1.add(tf.keras.layers.Dense(NUM_CATEGORIES, activation="softmax"))

#Model compilation
model1.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])


2024-01-16 22:24:16.909902: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2024-01-16 22:24:16.910170: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2024-01-16 22:24:16.910223: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (jupyterlab-velascosergi): /proc/driver/nvidia/version does not exist
2024-01-16 22:24:16.919416: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [16]:
model1.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 48, 48, 50)        1400      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 24, 24, 50)       0         
 )                                                               
                                                                 
 flatten (Flatten)           (None, 28800)             0         
                                                                 
 dropout (Dropout)           (None, 28800)             0         
                                                                 
 dense (Dense)               (None, 5)                 144005    
                                                                 
Total params: 145,405
Trainable params: 145,405
Non-trainable params: 0
__________________________________________________

In [17]:
model1.fit(images_train, labels_train, epochs = EPOCHS)

Epoch 1/10
422/422 [==============================] - 31s 71ms/step - loss: 12.2161 - accuracy: 0.4716
Epoch 2/10
422/422 [==============================] - 29s 68ms/step - loss: 0.8931 - accuracy: 0.6737
Epoch 3/10
422/422 [==============================] - 30s 71ms/step - loss: 0.6574 - accuracy: 0.7569
Epoch 4/10
422/422 [==============================] - 29s 68ms/step - loss: 0.5358 - accuracy: 0.8072
Epoch 5/10
422/422 [==============================] - 30s 71ms/step - loss: 0.4682 - accuracy: 0.8321
Epoch 6/10
422/422 [==============================] - 29s 69ms/step - loss: 0.4020 - accuracy: 0.8541
Epoch 7/10
422/422 [==============================] - 30s 71ms/step - loss: 0.4272 - accuracy: 0.8549
Epoch 8/10
422/422 [==============================] - 29s 69ms/step - loss: 0.3691 - accuracy: 0.8743
Epoch 9/10
422/422 [==============================] - 31s 73ms/step - loss: 0.3591 - accuracy: 0.8745
Epoch 10/10
422/422 [==============================] - 29s 69ms/step - loss: 0.36

In [18]:
model1.evaluate(images_test, labels_test)

47/47 [==============================] - 1s 12ms/step - loss: 1.6268 - accuracy: 0.6860


[1.6267576217651367, 0.6860387325286865]

In [19]:
model1.save("Model1")

INFO:tensorflow:Assets written to: Model1/assets


INFO:tensorflow:Assets written to: Model1/assets


<h3> Complex model: 3 convolution layer

Now we will use the basic model to add two internal layers and evaluate the new performance

In [20]:
model2 = tf.keras.models.Sequential()

#Create a Conv2D layer
model2.add(tf.keras.layers.Conv2D(50, (3,3), activation='relu', input_shape=(IMG_WIDTH, IMG_HEIGHT, 3)))

#Create a Pooling layer
model2.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))

 #2nd layer
model2.add(tf.keras.layers.Conv2D(50, (2,2), activation='relu'))
model2.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))

#3rd layer
model2.add(tf.keras.layers.Conv2D(50, (1,1), activation='relu'))
model2.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))

#Flatten layer
model2.add(tf.keras.layers.Flatten())

model2.add(tf.keras.layers.Dropout(0.05))

#Output layer
model2.add(tf.keras.layers.Dense(NUM_CATEGORIES, activation="softmax"))

#Model compilation
model2.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])

In [21]:
model2.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_1 (Conv2D)           (None, 48, 48, 50)        1400      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 24, 24, 50)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 23, 23, 50)        10050     
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 11, 11, 50)       0         
 2D)                                                             
                                                                 
 conv2d_3 (Conv2D)           (None, 11, 11, 50)        2550      
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 5, 5, 50)        

In [22]:
model2.fit(images_train, labels_train, epochs = EPOCHS)

Epoch 1/10
422/422 [==============================] - 32s 74ms/step - loss: 2.1480 - accuracy: 0.4682
Epoch 2/10
422/422 [==============================] - 30s 70ms/step - loss: 0.8766 - accuracy: 0.6730
Epoch 3/10
422/422 [==============================] - 31s 73ms/step - loss: 0.7315 - accuracy: 0.7252
Epoch 4/10
422/422 [==============================] - 30s 70ms/step - loss: 0.6741 - accuracy: 0.7402
Epoch 5/10
422/422 [==============================] - 31s 75ms/step - loss: 0.6195 - accuracy: 0.7664
Epoch 6/10
422/422 [==============================] - 30s 70ms/step - loss: 0.5782 - accuracy: 0.7814
Epoch 7/10
422/422 [==============================] - 31s 74ms/step - loss: 0.5588 - accuracy: 0.7889
Epoch 8/10
422/422 [==============================] - 30s 72ms/step - loss: 0.5287 - accuracy: 0.8031
Epoch 9/10
422/422 [==============================] - 31s 73ms/step - loss: 0.5078 - accuracy: 0.8007
Epoch 10/10
422/422 [==============================] - 30s 71ms/step - loss: 0.482

In [23]:
model2.evaluate(images_test, labels_test)

47/47 [==============================] - 1s 18ms/step - loss: 0.5221 - accuracy: 0.7943


[0.5221474766731262, 0.7942551970481873]

In [24]:
model2.save("Model2")

INFO:tensorflow:Assets written to: Model2/assets


INFO:tensorflow:Assets written to: Model2/assets


<h3> Loading ResNet network

In [15]:
#!pip install tensorflow

In [29]:
#del basemodel, x, model3

In [30]:
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.models import Model

# initialize the base model
basemodel = ResNet50(input_shape=(IMG_WIDTH,IMG_HEIGHT,3),
                          include_top = False,
                          weights = 'imagenet')

for layer in basemodel.layers:
    layer.trainable = False

#x = basemodel.output
    
#x = tf.keras.models.Sequential()
x = tf.keras.layers.Flatten()(basemodel.output)
x = tf.keras.layers.Dense(50, activation='relu')(x)
x = tf.keras.layers.Dropout(0.05)(x)

#Output layer
output_layer = tf.keras.layers.Dense(NUM_CATEGORIES, activation="softmax")(x)

model3 = Model(basemodel.input,output_layer)

model3.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])

/home/jupyterlab/conda/envs/python/lib/python3.7/site-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


In [ ]:
model3.summary()

In [34]:
model3.fit(images_train, labels_train, epochs = 1)

13474/13474 [==============================] - 536s 40ms/sample - loss: 0.3592 - acc: 0.8634


In [35]:
model3.evaluate(images_test, labels_test)

1497/1497 [==============================] - 48s 32ms/sample - loss: 1.0005 - acc: 0.6907


[1.000512338990598, 0.6907148]

In [36]:
model3.save('Model3.h5')